# Prediction number of stars for a review

Our dataset is quite large, about 6GB. For debugging our code, we will use [sample](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=sample#pyspark.sql.DataFrame.sample) after reading the JSON.

In [1]:
reviews_on_hdfs = "/user/qlr/data/yelp_academic_dataset_review.json"

In [2]:
reviews = spark.read.json(reviews_on_hdfs).sample(0.000001)
reviews.show(n=5)

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|wpSdYFSlHsW0h7pf4...|   1|2019-05-05 21:04:23|    1|Cccg84KTxjxp4iNBh...|  1.0|I scheduled a tim...|     1|qhHFB4StIDSxbbQvA...|
|sw0nkPQvtxLtTyRnr...|   1|2017-11-21 01:28:35|    0|UK6VRt5wsx1UkNe7Y...|  3.0|Service was good-...|     0|6xfUgjxHs7XZGwmwb...|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+



# Transforming Data

Spark has a vast library of feature engineering functions. For example, we can get TF-IDF representation for our review corpus. In the following snippet we construct a data preparation pipeline with three stages:
1. we get review text parsed into words
1. we count term frequencies of our bags of words
1. we normalise by inverted document frequency

In [3]:
%%time

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF

data_preparation = Pipeline(stages=[
    Tokenizer(inputCol="text", outputCol="words"),
    HashingTF(inputCol="words", outputCol="term_frequency"),
    IDF(inputCol="term_frequency", outputCol="embedding")
])
prepared_reviews = data_preparation.fit(reviews).transform(reviews)
prepared_reviews.select("text", "words", "term_frequency", "embedding").show(n=5)

+--------------------+--------------------+--------------------+--------------------+
|                text|               words|      term_frequency|           embedding|
+--------------------+--------------------+--------------------+--------------------+
|I scheduled a tim...|[i, scheduled, a,...|(262144,[4081,791...|(262144,[4081,791...|
|Service was good-...|[service, was, go...|(262144,[9639,156...|(262144,[9639,156...|
+--------------------+--------------------+--------------------+--------------------+

CPU times: user 56.1 ms, sys: 17.8 ms, total: 73.9 ms
Wall time: 24.7 s


Let's look into the details of the first row:

In [4]:
prepared_reviews.select("text", "words", "term_frequency", "embedding").head()

Row(text=u"I scheduled a time for them to move my belongings and asked that they call or text when they were in route. They didn't call until they were 3 hours late. They stated their system was down and they could do it the next day instead. That was not an option for me. I only had a small quantity of boxes and a few large items and had to ask them to please come. They agreed to another time slot, for which, they were late again. Once they arrived they whipped out a contract and listed a price 4xs greater than what we agreed on.  No negotiating with them. They asked for over $400 to move less than 20 small and medium boxes a bed n couch 2 miles away. Totally unprofessional, I do not recommend, awful experience had me in tears because I waited all day for them just to pull the bait n switch on me. It was 9 pm before they finally left", words=[u'i', u'scheduled', u'a', u'time', u'for', u'them', u'to', u'move', u'my', u'belongings', u'and', u'asked', u'that', u'they', u'call', u'or', u'

Mind the representation of TF-IDF vectors - it's sparse.

# Do It Yourself

Try to follow [a tutorial from Spark docs](http://spark.apache.org/docs/latest/ml-classification-regression.html#regression)

* calculate `word2vec` embeddings instead of TF-IDF
* build a linear regression (predict stars by text)
* split data into train and validation sets and evaluate your model
* compare quality of models (TF-IDF vs word2vec, linear vs random forest vs gradient goosted trees)

In [5]:
from pyspark.ml.feature import Word2Vec

In [6]:
data_preparation = Pipeline(stages=[
    Tokenizer(inputCol="text", outputCol="words"),
    Word2Vec(inputCol="words", outputCol="model")
])
prepared_reviews = data_preparation.fit(reviews).transform(reviews)
prepared_reviews.select("text", "words", "model").show(n=5)

+--------------------+--------------------+--------------------+
|                text|               words|               model|
+--------------------+--------------------+--------------------+
|I scheduled a tim...|[i, scheduled, a,...|[7.03314652465685...|
|Service was good-...|[service, was, go...|[4.08442619137783...|
+--------------------+--------------------+--------------------+

